# RSS Demo

`rss_to_sqlite.py` is a Python script that ingests RSS feed URLs, parses their content, stores the data in an SQLite database, and provides an option to export the data to a CSV file.

## Functions

- **`fetch_and_parse_rss(feed_urls)`**: Takes a list of RSS feed URLs, fetches and parses the feed data, and returns a list of entries containing the title, link, description, and publication date.

- **`save_to_database(entries, db_path='rss_data.db')`**: Saves the parsed RSS feed entries into an SQLite database. Creates a new database if one does not exist.

- **`export_to_csv(db_path='rss_data.db', csv_path='rss_data.csv')`**: Exports the data from the SQLite database to a CSV file.

In [1]:
import feedparser
import sqlite3
import csv

def fetch_and_parse_rss(feed_urls):
    """
    Fetch and parse RSS feeds from the provided URLs.
    """
    entries = []
    for url in feed_urls:
        feed = feedparser.parse(url)
        for entry in feed.entries:
            entries.append({
                'title': entry.title,
                'link': entry.link,
                'description': entry.get('description', ''),
                'published': entry.get('published', '')
            })
    return entries

def save_to_database(entries, db_path='rss_data.db'):
    """
    Save parsed RSS feed entries into an SQLite database.
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    # Create table if it does not exist
    c.execute('''CREATE TABLE IF NOT EXISTS rss_entries
                 (id INTEGER PRIMARY KEY, title TEXT, link TEXT, description TEXT, published TEXT)''')
    
    # Insert data into the table
    for entry in entries:
        c.execute('''INSERT INTO rss_entries (title, link, description, published)
                     VALUES (?, ?, ?, ?)''', (entry['title'], entry['link'], entry['description'], entry['published']))
    
    conn.commit()
    conn.close()

def export_to_csv(db_path='rss_data.db', csv_path='rss_data.csv'):
    """
    Export data from the SQLite database to a CSV file.
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    # Fetch all data from the table
    c.execute('SELECT * FROM rss_entries')
    rows = c.fetchall()

    # Write data to CSV
    with open(csv_path, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['ID', 'Title', 'Link', 'Description', 'Published'])
        writer.writerows(rows)

    conn.close()

if __name__ == "__main__":
    # Example RSS feed URLs
    feed_urls = [
        'https://rss.cnn.com/rss/cnn_topstories.rss',
        'https://rss.nytimes.com/services/xml/rss/nyt/HomePage.xml'
    ]
    
    # Fetch and parse the RSS feeds
    entries = fetch_and_parse_rss(feed_urls)
    
    # Save parsed entries to SQLite database
    save_to_database(entries)
    
    # Export the data from SQLite to CSV
    export_to_csv()

In [8]:
import feedparser
import requests
from bs4 import BeautifulSoup

# Step 1: Parse the RSS feed
feed_url = 'https://apnews.com/news-sitemap-content.xml'  # Replace with your RSS feed URL
feed = feedparser.parse(feed_url)

# Step 2: Loop through each article entry in the feed
for entry in feed.entries:
    article_url = entry.link  # Get the article URL from the feed
    print(f"Fetching article: {entry.title} from {article_url}")

    # Step 3: Download the HTML content of the article
    response = requests.get(article_url)
    
    # Check if the request was successful
    if response.status_code == 200:
        html_content = response.content  # Get the raw HTML content
        
        # Step 4: Parse the HTML using BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Example: Find the main article body using common tags
        # You may need to adjust the selector based on the actual HTML structure
        article_body = soup.find('div', class_='article-body')  # Adjust tag/class based on actual HTML
        if article_body:
            full_text = article_body.get_text(strip=True)  # Extract the text content
            print(f"Article content:\n{full_text}\n")
        else:
            print("Could not find the article body. Adjust the selector based on the website's structure.")
    else:
        print(f"Failed to fetch the article. Status code: {response.status_code}")


# query_database Function

The `query_database` function connects to an SQLite database and executes a query to fetch all entries from the `rss_entries` table.

## Functionality

- **Connect to the Database**: Opens a connection to the specified SQLite database (`rss_data.db` by default).
- **Execute SQL Query**: Runs a query to fetch all records from the `rss_entries` table.
- **Fetch and Display Results**: Retrieves all rows returned by the query and prints them to the console.
- **Close the Connection**: Ensures that the database connection is closed after the query is executed.

## Usage

To run the function, simply execute the script. It will display all entries from the `rss_entries` table in the SQLite database.

### Example

```python
if __name__ == "__main__":
    query_database()

In [2]:
import sqlite3

def query_database(db_path='rss_data.db'):
    """
    Connect to the SQLite database and execute a query to fetch all entries.
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    # Basic query: Fetch all records
    c.execute('SELECT * FROM rss_entries')
    
    # Fetch all results
    rows = c.fetchall()
    
    # Print the results
    for row in rows:
        print(row)
    
    conn.close()

if __name__ == "__main__":
    query_database()

(1, 'For Trump, Tariffs Are the Solution to Almost Any Problem', 'https://www.nytimes.com/2024/09/10/us/politics/trump-enthusiasm-tariffs.html', 'The former president has proposed using tariffs to fund child care, boost manufacturing, quell immigration and encourage use of the dollar. Economists are skeptical.', 'Tue, 10 Sep 2024 19:04:25 +0000')
(2, 'Harris Economic Plan Focuses on Prices, a Key Vulnerability', 'https://www.nytimes.com/2024/09/10/us/politics/kamala-harris-economy-prices.html', 'Vice President Kamala Harris has been balancing the challenges of defending “Bidenomics” and charting her own course on the economy.', 'Tue, 10 Sep 2024 13:54:35 +0000')
(3, 'Trump’s Promises to Cut Inflation Are Unrealistic, Many Economists Say', 'https://www.nytimes.com/2024/09/10/us/politics/trump-economics-gas-tariffs-inflation.html', 'Economists and analysts are dubious of Trump’s promises to slash gas prices or prod interest rates lower.', 'Tue, 10 Sep 2024 17:12:29 +0000')
(4, 'Fearing A

In [3]:
import sqlite3

def query_database(db_path='rss_data.db'):
    """
    Connect to the SQLite database and execute a query to fetch entries containing
    the words 'energy' or 'gas' in the title or description.
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    # Query: Fetch records where title or description contains 'energy' or 'gas'
    c.execute('''
        SELECT * FROM rss_entries 
        WHERE title LIKE '%energy%' OR title LIKE '%gas%'
        OR description LIKE '%energy%' OR description LIKE '%gas%'
    ''')
    
    # Fetch all results
    rows = c.fetchall()
    
    # Print the results
    for row in rows:
        print(row)
    
    conn.close()

if __name__ == "__main__":
    query_database()

(3, 'Trump’s Promises to Cut Inflation Are Unrealistic, Many Economists Say', 'https://www.nytimes.com/2024/09/10/us/politics/trump-economics-gas-tariffs-inflation.html', 'Economists and analysts are dubious of Trump’s promises to slash gas prices or prod interest rates lower.', 'Tue, 10 Sep 2024 17:12:29 +0000')


In [4]:
import sqlite3
import json

def export_to_json(db_path='rss_data.db', output_file='rss_data.json'):
    """
    Connect to the SQLite database, fetch data, and export it to a JSON file.
    """
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    
    # Query: Fetch all records
    c.execute('SELECT * FROM rss_entries')
    
    # Fetch all results
    rows = c.fetchall()
    
    # Get column names
    column_names = [description[0] for description in c.description]
    
    # Convert the fetched data into a list of dictionaries
    data = [dict(zip(column_names, row)) for row in rows]
    
    # Export data to a JSON file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    
    conn.close()
    print(f"Data has been exported to {output_file}.")

if __name__ == "__main__":
    export_to_json()

Data has been exported to rss_data.json.
